In [9]:
# pip install openpyxl

In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_excel('data/Color_Game_Endo.xlsx')
df.head()

,session_code_n,id_in_group,round_number,color_n,score,actionType_n,new_color_n,delete_player_n,add_player_n,linked_neighbors
0,5,1,1,4,0,4,996,996,996,"2, 3, 4, 16, 17, 18"
1,5,2,1,4,0,1,3,996,996,"1, 3, 4, 5, 17, 18"
2,5,3,1,4,0,1,3,996,996,"1, 2, 4, 5, 6, 18"
3,5,4,1,4,0,1,1,996,996,"1, 2, 3, 5, 6, 7"
4,5,5,1,4,0,1,1,996,996,"2, 3, 4, 6, 7, 8"


### Dataset說明
* 相同session: 相同組user
* 相同round: 同一個graph -> 某個時期的狀態
* node feature: id, color, score, history(each action %), neighbor_num
* actionType = ground truth

session + user_id + round 可以決定一個node (primary key)

---

把action == 7刪除 (結束)，用session、round、id重新排序

In [12]:
df = df.sort_values(['session_code_n', 'round_number', 'id_in_group'])
df = df.drop(df[df['actionType_n']==7].index)
df = df.reset_index(drop=True)

合併action
* 每兩個分別為有 / 無 timeout 的相同 action，共分成三種

In [13]:
df.loc[df[df['actionType_n']==1].index, 'actionType_n'] = 0
df.loc[df[df['actionType_n']==2].index, 'actionType_n'] = 0
df.loc[df[df['actionType_n']==3].index, 'actionType_n'] = 1
df.loc[df[df['actionType_n']==4].index, 'actionType_n'] = 1
df.loc[df[df['actionType_n']==5].index, 'actionType_n'] = 2
df.loc[df[df['actionType_n']==6].index, 'actionType_n'] = 2

color從0開始做index

In [14]:
df.loc[df[df['new_color_n']==1].index, 'new_color_n'] = 0
df.loc[df[df['new_color_n']==2].index, 'new_color_n'] = 1
df.loc[df[df['new_color_n']==3].index, 'new_color_n'] = 2
df.loc[df[df['new_color_n']==4].index, 'new_color_n'] = 3

df.loc[df[df['color_n']==1].index, 'color_n'] = 0
df.loc[df[df['color_n']==2].index, 'color_n'] = 1
df.loc[df[df['color_n']==3].index, 'color_n'] = 2
df.loc[df[df['color_n']==4].index, 'color_n'] = 3

加入graph size

In [15]:
df['graph_size'] = 0
session_num = 14
session = [i for i in range(1, session_num)]
for s in session:
    session_df = df[df['session_code_n'] == s]
    graph_size = len(session_df.drop_duplicates('id_in_group'))
    df['graph_size'].iloc[session_df.index] = graph_size

C:\Users\user\AppData\Local\Temp\ipykernel_17312\1238754407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['graph_size'].iloc[session_df.index] = graph_size


加入history information

In [16]:
df['hist_color'] = 0
df['hist_neighbor'] = 0
df['hist_skip'] = 0

graph_size = list(df.drop_duplicates('session_code_n')['graph_size'])
session_and_size_dict = {session[i]: graph_size[i] for i in range(len(session))}

# 1 2: color, 3 4: skip, 5 6: player
for s in session:
    session_df = df[df['session_code_n'] == s]
    user_hist = {i:[0, 0, 0] for i in range(1, session_and_size_dict[s]+1)}
    # print(len(user_hist))
    for idx, row in session_df.iterrows():
        id = row['id_in_group']
        round = row['round_number']-1
        action = row['actionType_n']
        if(round != 0):
            df.loc[idx, 'hist_color'] = np.round(user_hist[id][0] / round, 2)
            df.loc[idx, 'hist_skip'] = np.round(user_hist[id][1] / round, 2)
            df.loc[idx, 'hist_neighbor'] = np.round(user_hist[id][2] / round, 2)
        else:
            df.loc[idx, 'hist_color'] = np.round(user_hist[id][0], 2)
            df.loc[idx, 'hist_skip'] = np.round(user_hist[id][1], 2)
            df.loc[idx, 'hist_neighbor'] = np.round(user_hist[id][2], 2)


        if(action == 0):
            user_hist[id][0] += 1
        elif(action == 1):
            user_hist[id][1] += 1
        elif(action == 2):
            user_hist[id][2] += 1

In [17]:
df['num_of_neighbor'] = [len(str(neighbor_list).split(',')) for neighbor_list in df['linked_neighbors']]

In [18]:
df

,session_code_n,id_in_group,round_number,color_n,score,actionType_n,new_color_n,delete_player_n,add_player_n,linked_neighbors,graph_size,hist_color,hist_neighbor,hist_skip,num_of_neighbor
0,1,1,1,3,0,0,0,996,996,"2,3,4,13,14,15",15,0.0,0.0,0.0,6
1,1,2,1,3,0,0,1,996,996,"1,3,4,5,14,15",15,0.0,0.0,0.0,6
2,1,3,1,3,0,0,0,996,996,"1,2,4,5,6,15",15,0.0,0.0,0.0,6
3,1,4,1,3,0,0,1,996,996,"1,2,3,5,6,7",15,0.0,0.0,0.0,6
4,1,5,1,3,0,0,1,996,996,"2,3,4,6,7,8",15,0.0,0.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,13,16,6,1,100,1,996,996,996,"13, 14, 15, 17, 18, 8, 19",20,0.2,0.0,0.8,7
2149,13,17,6,2,100,1,996,996,996,"14, 15, 16, 18, 19, 4, 6",20,0.2,0.2,0.6,7
2150,13,18,6,0,89,2,996,1,13,"1, 15, 16, 17, 19, 20, 7, 5, 11",20,0.4,0.0,0.6,9
2151,13,19,6,3,100,1,996,996,996,"2, 17, 18, 4, 16, 5, 6",20,0.0,0.6,0.4,7


---
分成node feature和graph sturcture
* graph feature: session, id, round, size, color, score, num_of_neighbor, hist_color, hist_neighbor, hist_skip
* graph structure: session, id, round, size, linked

In [19]:
struct = {
    'session': df['session_code_n'], 
    'uid': df['id_in_group'], 
    'round': df['round_number'], 
    'linked': df['linked_neighbors']
}
graph_struct_df = pd.DataFrame(struct)
graph_struct_df.head()

,session,uid,round,linked
0,1,1,1,"2,3,4,13,14,15"
1,1,2,1,"1,3,4,5,14,15"
2,1,3,1,"1,2,4,5,6,15"
3,1,4,1,"1,2,3,5,6,7"
4,1,5,1,"2,3,4,6,7,8"


In [20]:
feature = {
    'session': df['session_code_n'], 
    'uid': df['id_in_group'], 
    'round': df['round_number'], 
    'color': df['color_n'], 
    'score': df['score'], 
    'num_of_neighbor': df['num_of_neighbor'], 
    'hist_color': df['hist_color'], 
    'hist_neighbor': df['hist_neighbor'], 
    'hist_skip': df['hist_skip'], 
    'action': df['actionType_n']
}
graph_feature_df = pd.DataFrame(feature)
graph_feature_df.head()

,session,uid,round,color,score,num_of_neighbor,hist_color,hist_neighbor,hist_skip,action
0,1,1,1,3,0,6,0.0,0.0,0.0,0
1,1,2,1,3,0,6,0.0,0.0,0.0,0
2,1,3,1,3,0,6,0.0,0.0,0.0,0
3,1,4,1,3,0,6,0.0,0.0,0.0,0
4,1,5,1,3,0,6,0.0,0.0,0.0,0


In [21]:
df.to_csv('data/processed_data.csv', index=False)
graph_struct_df.to_csv('data/graph_structure.csv', index=False)
graph_feature_df.to_csv('data/graph_feature.csv', index=False)

---

focus on color change round

(不能把沒選color的node刪除 -> 他們也會提供其他user資訊!)
* 只能都留著，在訓練的時候只看new_color != 996 的 node

In [22]:
color_df = df

In [23]:
feature = {
    'session': color_df['session_code_n'], 
    'uid': color_df['id_in_group'], 
    'round': color_df['round_number'], 
    'color': color_df['color_n'], 
    'score': color_df['score'], 
    'num_of_neighbor': color_df['num_of_neighbor'], 
    'hist_color': color_df['hist_color'], 
    'hist_neighbor': color_df['hist_neighbor'], 
    'hist_skip': color_df['hist_skip'], 
    'new_color': color_df['new_color_n']
}
color_df = pd.DataFrame(feature)
color_df.head()

,session,uid,round,color,score,num_of_neighbor,hist_color,hist_neighbor,hist_skip,new_color
0,1,1,1,3,0,6,0.0,0.0,0.0,0
1,1,2,1,3,0,6,0.0,0.0,0.0,1
2,1,3,1,3,0,6,0.0,0.0,0.0,0
3,1,4,1,3,0,6,0.0,0.0,0.0,1
4,1,5,1,3,0,6,0.0,0.0,0.0,1


In [24]:
color_df.to_csv('data/change_color_df.csv', index=False)

---

focus on change neighbor

In [25]:
neighbor_df = df
feature = {
    'session': neighbor_df['session_code_n'], 
    'uid': neighbor_df['id_in_group'], 
    'round': neighbor_df['round_number'], 
    'color': neighbor_df['color_n'], 
    'score': neighbor_df['score'], 
    'num_of_neighbor': neighbor_df['num_of_neighbor'], 
    'hist_color': neighbor_df['hist_color'], 
    'hist_neighbor': neighbor_df['hist_neighbor'], 
    'hist_skip': neighbor_df['hist_skip'], 
    'del_neighbor': neighbor_df['delete_player_n'], 
    'add_neighbor': neighbor_df['add_player_n']
}

neighbor_df = pd.DataFrame(feature)
neighbor_df.head()

,session,uid,round,color,score,num_of_neighbor,hist_color,hist_neighbor,hist_skip,del_neighbor,add_neighbor
0,1,1,1,3,0,6,0.0,0.0,0.0,996,996
1,1,2,1,3,0,6,0.0,0.0,0.0,996,996
2,1,3,1,3,0,6,0.0,0.0,0.0,996,996
3,1,4,1,3,0,6,0.0,0.0,0.0,996,996
4,1,5,1,3,0,6,0.0,0.0,0.0,996,996


In [26]:
neighbor_df.to_csv('data/change_neighbor_df.csv', index=False)